# Принятие решений в бизнесе. Анализ интернет-магазина.

### Описание данных

**Входные данные**


*    список гипотез (файл `hypothesis.csv`)

`Hypothesis` — краткое описание гипотезы;   
`Reach` — охват пользователей по 10-балльной шкале;   
`Impact` — влияние на пользователей по 10-балльной шкале;   
`Confidence` — уверенность в гипотезе по 10-балльной шкале;    
`Efforts` — затраты ресурсов на проверку гипотезы по 10-балльной шкале.   

 
*    информация о заказах пользователей в интернет-магазине (файл `orders.csv.`)    

`transactionId` — идентификатор заказа;    
`visitorId` — идентификатор пользователя, совершившего заказ;    
`date` — дата, когда был совершён заказ;    
`revenue` — выручка заказа;    
`group` — группа A/B-теста, в которую попал заказ.    

 
*    информация о посещениях пользователями интернет-магазина (файл `visitors.csv`)

`date` — дата;   
`group` — группа A/B-теста;    
`visitors` — количество пользователей в указанную дату в указанной группе A/B-теста.    
 
 
 
**Ход исследования**

Исследование пройдёт в три этапа:

*    Обзор  и предобработка данных;
*    Приоритезация гипотез;
*    Анализ А/В-теста;

### Обзор данных

In [ ]:
pip install missingno

In [ ]:
#импортируем необходимые библиотеки 
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import math
from scipy import stats as st

In [ ]:
hypothesis = pd.read_csv('/datasets/hypothesis.csv')
orders = pd.read_csv('/datasets/orders.csv')
visitors = pd.read_csv('/datasets/visitors.csv') #загрузим данные 

In [ ]:
hypothesis.info()
hypothesis

In [ ]:
# Проверим датафрейм на наличие дубликатов
print('Количество явных дубликатов во датафреймах:')
print('hypothesis - ', hypothesis.duplicated().sum())

In [ ]:
msno.matrix(hypothesis, labels=True) #оценим визуально пропуски в данных 

Выводы:

Мы видим в данных 9 различных гипотез. У каждой по 4ём критериям оценки от 1 до 10. Здесь всё хорошо
некорректные названия столбцов (заглавные буквы)
пропусков нет, дубликатов нет, типы данных корректны.

In [ ]:
orders.info()
orders.head(10)

In [ ]:
# Проверим датафрейм на наличие дубликатов
print('Количество явных дубликатов во датафреймах:')
print('orders - ', orders.duplicated().sum())

In [ ]:
msno.matrix(orders, labels=True) #оценим визуально пропуски в данных 

Выводы:
- некорректные названия столбцов
- некорреткный тип данных у столбца `date`
- пропусков нет
- явных дубликатов нет

In [ ]:
visitors.info()
visitors.head(10)

In [ ]:
# Проверим датафрейм на наличие дубликатов
print('Количество явных дубликатов во датафреймах:')
print('visitors - ', visitors.duplicated().sum())

In [ ]:
msno.matrix(visitors, labels=True) #оценим визуально пропуски в данных 

Выводы:
- некорректный тип данных столбца `date`
- пропусков нет
- проверить на дубликаты( полные и неочевидные)

### Предобработка данных

Переименуем столбцы в таблицах

In [ ]:
hypothesis.columns = ['hypothesis','reach','impact','confidence', 'efforts']
orders.columns = ['transaction_id','visitor_id', 'date','revenue','group']

Приведём столбцы с датами к корректному типу данных 

In [ ]:
orders['date'] = pd.to_datetime(orders['date'])
visitors['date'] = pd.to_datetime(visitors['date'])

Привели данные к хорошему стилю, изменили типы данных для дальнейшей работы

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Данные получены и подготовлены к анализу! 
</div>

Посчитаем количество уникальных пользоваталей в нашей A/B тесте

In [ ]:
len(orders['visitor_id'].unique())

А теперь посчитаем по группам:

In [ ]:
group_A = orders.query('group == "A"')
len(group_A['visitor_id'].unique())

In [ ]:
group_B = orders.query('group == "B"')
len(group_B['visitor_id'].unique())

В общей таблице у нас 1031 уникальных пользоваталей. А при разделении на группы 503+586 = 1089. То есть есть 58 пользователей(1089-1031), которые попали в обе группы. Найдем их 

In [ ]:
union = group_A.merge(group_B,on = 'visitor_id')
len(union['visitor_id'].unique())

Нашли этих пользователей. Мы не можем определить в какую группу они должны были попасть. Чтобы не угадывать - просто удалим этих пользователей из первоначальной таблицы

In [ ]:
visitor_both_group = union['visitor_id'].to_list()
orders = orders[~orders['visitor_id'].isin(visitor_both_group)]
len(orders['visitor_id'].unique())

Было 1031 уникальный, осталось 973. Удалили 58 пользоваталей, которые попали в обе группы 

### Приоритезация гипотез

Когда рассчитываеют гипотезы, обычно используют статистические методы, такие как тестирование гипотез, чтобы определить, является ли различие между группами или переменными статистически значимым. Один из инструментов, используемых для этого, - это p-значение, которое показывает вероятность того, что наблюдаемые данные могут быть получены случайно.

Ранее мы увидели, что отдел Маркетинга подготовил список из 9-ти гипотез, каждый компонент которой оценен по шкале от 0 до 10.

Применим один из самых популярных методов приоритезации гипотез - ICE (от. англ.: Impact, Confidence, Effort/ Влияние, Уверенность, Усилия).

In [ ]:
pd.options.display.max_colwidth = 400

In [ ]:
hypothesis['ice'] = hypothesis['impact'] * hypothesis['confidence'] / hypothesis['efforts']

hypothesis[['hypothesis', 'ice']].sort_values(by='ice', ascending=False).head(5)

In [ ]:
ax = (hypothesis[['hypothesis', 'ice']]
      .set_index('hypothesis')
      .sort_values('ice', ascending=False)
      .plot(kind='barh', color='g', alpha=0.4)
     )
ax.grid(True)
ax.set_xlabel('Приоритет')
ax.set_ylabel(' ')
ax.set_title('Приоритезация гипотез по методу ICE', loc='left')
plt.show()

Исходя из данных мы видим топ-5 гипотез, которым следует уделить пристальное внимание. Добавим к расчету параметр Reach (англ.: Охват) и приоритезируем гипотезы по методу RICE.

In [ ]:
hypothesis['rice'] = hypothesis['reach'] * hypothesis['impact'] * hypothesis['confidence'] / hypothesis['efforts']

hypothesis[['hypothesis', 'rice']].sort_values(by='rice', ascending=False).head(5)

In [ ]:
ax = (hypothesis[['hypothesis', 'rice']]
      .set_index('hypothesis')
      .sort_values('rice', ascending=False)
      .plot(kind='barh', color='g', alpha=0.4)
     )
ax.grid(True)
ax.set_xlabel('Приоритет')
ax.set_ylabel(' ')
ax.set_title('Приоритезация гипотез по методу RICE', loc='left')
plt.show()

Мы видим те же самые гипотезы, только в другой последовательности. Разница приоритезиций заключается в том, что одна учитывает кол-во пользователей, которых затронут изменения, а другая - нет.

In [ ]:
hypothesis[['hypothesis','ice', 'rice']].sort_values(by='rice', ascending=False).head(5)

По фреймворку ICE наибольшую оценку имели гпиотезы 8,0 и 7. После применения фреймворка RICE приоритеты изменились на гпиотезы 7,2 и 0. Это объясняется тем, что в случае фреймворка ICE в отличие от RICE мы не учитывали охват изменений, тогда как это важная составляющая оценки гипотез.

В нашем случае наиболее приоритетными гипотезами будут 7, 2, 0.

### Анализ A/B-теста

Подготовим таблицу для анализа результатов A/B теста. Посчитаем средний чек и конверсию по группам/по дням

In [ ]:
orders_grouped = (orders.groupby(['date','group'], as_index=False)
                        .agg({'transaction_id':'nunique','visitor_id':'nunique','revenue':'sum'}))\
                        .rename(columns={'transaction_id':'orders','visitor_id':'buyers'})
orders_grouped['group_copy'] = orders_grouped['group']
orders_grouped = orders_grouped.set_index(['date','group'])
orders_cumsum = orders_grouped.sort_values(['date','group']).groupby('group_copy').cumsum().reset_index()

In [ ]:
orders_grouped

In [ ]:
# посчитаем сумму визитов по дням в каждой группе
visitors_cumsum = visitors.groupby(['date','group'], as_index=False).agg({'visitors':'sum'}).sort_values(['date','group'])
# скопируем group чтобы провести по ней кумулятивную сумму
visitors_cumsum['group_copy'] = visitors_cumsum['group']
visitors_cumsum = visitors_cumsum.set_index(['date','group']).groupby('group_copy').cumsum().reset_index()    

In [ ]:
cummulative = orders_cumsum.join(visitors_cumsum[['visitors']])

In [ ]:
#считаем средний чек и конверсию 
cummulative['average_check'] = cummulative['revenue'] / cummulative['orders']
cummulative['conversion'] = cummulative['orders'] / cummulative['visitors']

In [ ]:
cummulative.head()

Посчитали кумулятивные данные по группам и дням( количество заказов, покупателей, выручку, посетителей). А так же средний чек и конверсию. Теперь можно приступать к тестам.

#### График кумулятивной выручки по группам

In [ ]:
cummulativeA = cummulative[cummulative['group']=='A'][['date', 'revenue']]
cummulativeB = cummulative[cummulative['group']=='B'][['date', 'revenue']]
plt.plot(cummulativeA.iloc[:,0].to_frame(), cummulativeA.iloc[:,1].to_frame(), label='A' )
plt.plot(cummulativeB.iloc[:,0].to_frame(), cummulativeB.iloc[:,1].to_frame(), label='B')
plt.title('График кумулятивной выручки по группам')
plt.ylabel("Выручка")
plt.xticks(rotation=30)
plt.legend()
plt.show()

Мы видим, что кумулятивная выручка группы А растет линейно,  и она меньше, чем у группы В.    
При этом в группе В есть резкий скачок в районе 18-го августа, что может сигнализировать о всплесках числа заказов, либо о появлении очень дорогих заказов в выборке. 

#### График кумулятивного среднего чека по группам

In [ ]:
cummulativeA = cummulative[cummulative['group']=='A'][['date', 'average_check']]
cummulativeB = cummulative[cummulative['group']=='B'][['date', 'average_check']]
plt.plot(cummulativeA.iloc[:,0].to_frame(), cummulativeA.iloc[:,1].to_frame(), label='A' )
plt.plot(cummulativeB.iloc[:,0].to_frame(), cummulativeB.iloc[:,1].to_frame(), label='B')
plt.title('График кумулятивного среднего чека по группам')
plt.ylabel("Выручка")
plt.xticks(rotation=30)
plt.legend()
plt.show()

Мы видим, что средний чек в группе А  через какое-то время стабилизировался. В тот же самый момент у группы B резкий скачок по кумулятивному среднему чеку, что говорит в пользу версии о дорогой покупке.

#### График относительного изменения кумулятивного среднего чека группы B к группе A

In [ ]:
temp = (cummulativeA[['date', 'average_check']].merge(cummulativeB[['date', 'average_check']], 
                                            left_on='date', right_on='date', how='left', suffixes=['_A', '_B']))
plt.plot(temp['date'], temp['average_check_B'] / temp['average_check_A']-1)
plt.title('График относительного изменения кумулятивного среднего чека группы B к группе A')
plt.axhline(y=0, color='black', linestyle='--')
plt.xticks(rotation=30)
plt.show()

Этот график так же иллюстируют такую же картину. Средний чек по группе B выше, чем по группе A, возможно есть крупные чеки, которые провоцируют выбросы.

#### График кумулятивной конверсии по группам

In [ ]:
cummulativeA = cummulative[cummulative['group']=='A'][['date', 'conversion']]
cummulativeB = cummulative[cummulative['group']=='B'][['date', 'conversion']]
plt.plot(cummulativeA.iloc[:,0].to_frame(), cummulativeA.iloc[:,1].to_frame(), label='A' )
plt.plot(cummulativeB.iloc[:,0].to_frame(), cummulativeB.iloc[:,1].to_frame(), label='B')
plt.title('График кумулятивной конверсии по группам')
plt.ylabel("Выручка")
plt.ylim(0.001)
plt.xticks(rotation=30)
plt.legend()
plt.show()

Сначала конверсии колебались, но довольно быстро выровнялись; результат группы В кажется более привлекательным.

#### График относительного изменения кумулятивной конверсии группы B к группе A

In [ ]:
temp = (cummulativeA[['date', 'conversion']].merge(cummulativeB[['date', 'conversion']], 
                                            left_on='date', right_on='date', how='left', suffixes=['_A', '_B']))
plt.plot(temp['date'], temp['conversion_B'] / temp['conversion_A']-1)
plt.title('График относительного изменения кумулятивной конверсии группы B к группе A')
plt.axhline(y=0, color='black', linestyle='--')
plt.xticks(rotation=45)
plt.show()

В начале теста группа В проигрывала группе А, но затем вырвалась вперед. Ее конверсия стремительно росла, далее начался медленный спад. Теперь конверсия группы В снова растет.   
Но мы помним, что графики выше сигнализировали нам о наличии крупных заказов.


#### График количества заказов по пользователям

Пользователи, совершившие много заказов влияют на отношение количества заказов к количеству поситителей интернет-магазина за время теста.    

"Обычный" пользователь редко совершает более одного-двух заказов в короткий срок (если только речь не идет о сайтах с регулярным спросом (например, продуктовый интернет-магазин,)). Посмотрим, что с количеством заказов происходит с участниками нашего тестирования.

In [ ]:
#создадим таблицу по количеству заказов каждого пользователя 
orders_per_user = orders.groupby('visitor_id', as_index=False).agg({'transaction_id':'nunique'})\
                         .rename(columns={'transaction_id':'orders'})
orders_per_user.head()

In [ ]:
x_values = pd.Series(range(0,len(orders_per_user['orders'])))
plt.scatter(x_values, orders_per_user['orders'], color='red', alpha = 0.2)
plt.show()

Много пользователей с 2-3 заказами. Их точная доля не ясна, поэтому сложно понять, можно ли считать их выбросами или нет.    
Посчитаем 95-й и 99-й перцентили количества заказов на пользователя и выберем границу для определения аномальных пользователей.

#### 95-й и 99-й перцентили количества заказов на пользователя. 

In [ ]:
print('95 перцентиль: %.2f' % (np.percentile(orders_per_user['orders'], [95])))
print('99 перцентиль: %.2f' % (np.percentile(orders_per_user['orders'], [99])))

Не более 5% пользователей совершали больше 2-х покупок в течение тестирования. И только 1% - четыре и более.   
Примем за верхнюю границу **3 заказа** на одного пользователя.

In [ ]:
#введем список без аномальных заказов
normal_orders = orders_per_user[orders_per_user['orders'] >= 3]['visitor_id'].to_list()

#### График стоимостей заказов

In [ ]:
x_values = pd.Series(range(0,len(orders['revenue'])))
plt.scatter(x_values, orders['revenue'], color='red',alpha = 0.1)
plt.show()

In [ ]:
display(orders['revenue'].describe())

# строим гистограмму
plt.hist(orders['revenue'], alpha=0.5) 
plt.title('Распределение стоимости заказов по пользователям')
plt.xlabel('Стоимость заказов')
plt.ylabel('Количество пользователей')
plt.show()

Опять мы видим этот гигантский заказ(почти 1.3 млн.!), который вызвал всплеск на графиках кумулятивных метрик, а так же другие весомые заказы, которые могут повлиять на результат исследований.

In [ ]:
#отфильтруем и построим график без двух очень крупных выбросов
orders_without_big_check = orders.query('revenue < 200000')
x_values = pd.Series(range(0,len(orders_without_big_check['revenue'])))
plt.scatter(x_values, orders_without_big_check['revenue'], color='red',alpha = 0.1)
plt.show()

Можем сказать,что среди остальные заказов так же есть несколько крупных выделяющихся (более 60000). Основная масса заказов не привышает 20000.

#### 95-й и 99-й перцентили стоимости заказов

In [ ]:
print('95 перцентиль: %.2f' % (np.percentile(orders['revenue'], [95])))
print('99 перцентиль: %.2f' % (np.percentile(orders['revenue'], [99])))

Определяем границу по величине 99% перцентиля (53904)

#### Cтатистическая значимость различий в конверсии между группами по «сырым» данным

Ранее мы результаты A/B-теста визуально и выяснили, что в данных, скорее всего, есть выбросы. Потом увидели выбросы и нашли границу для их определения.

Посчитаем статистическую значимость различий в конверсии между группами по «сырым» данным — без удаления аномальных пользователей.

<p>Введем основную и альтернативные гипотезы:</p>

$\begin{equation*}
 \begin{cases}
   H_0 :\text{различий в конверсии между группами нет}\\
   H_1 :\text{различия в конверсии между группами есть}
 \end{cases}
\end{equation*}$

Уровень значимости: $\alpha = 0.05$ 




Так как данные о продажах и средних чеках редко бывают нормально распределены (это классический пример переменных, подверженных выбросам), для проверки гипотез будем использовать непараметрический тест Уилкоксона-Манна-Уитни.   
Для уровеня значимости установим стандартный уровень статистической значимости, равный 0.05.

In [ ]:
visitors_daily_a = visitors[visitors['group'] == 'A'][['date', 'visitors']]
visitors_daily_a.columns = ['date', 'visitors_per_date_a']

visitors_cummulative_a = visitors_daily_a.apply(
    lambda x: visitors_daily_a[visitors_daily_a['date'] <= x['date']].agg(
        {'date': 'max', 'visitors_per_date_a': 'sum'}
    ),
    axis=1,
)
visitors_cummulative_a.columns = ['date', 'visitors_cummulative_a']

visitors_daily_b = visitors[visitors['group'] == 'B'][['date', 'visitors']]
visitors_daily_b.columns = ['date', 'visitors_per_date_b']

visitors_cummulative_b = visitors_daily_b.apply(
    lambda x: visitors_daily_b[visitors_daily_b['date'] <= x['date']].agg(
        {'date': 'max', 'visitors_per_date_b': 'sum'}
    ),
    axis=1,
)
visitors_cummulative_b.columns = ['date', 'visitors_cummulative_b']

orders_daily_a = (
    orders[orders['group'] == 'A'][['date', 'transaction_id', 'visitor_id', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transaction_id': pd.Series.nunique, 'revenue': 'sum'})
)
orders_daily_a.columns = ['date', 'orders_daily_a', 'revenue_per_date_a']

orders_cummulative_a = orders_daily_a.apply(
    lambda x: orders_daily_a[orders_daily_a['date'] <= x['date']].agg(
        {'date': 'max', 'orders_daily_a': 'sum', 'revenue_per_date_a': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])

orders_cummulative_a.columns = ['date','orders_cummulative_a','revenue_cummulative_a']

orders_daily_b = (
    orders[orders['group'] == 'B'][['date', 'transaction_id', 'visitor_id', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transaction_id': pd.Series.nunique, 'revenue': 'sum'})
)
orders_daily_b.columns = ['date', 'orders_daily_b', 'revenue_per_date_b']

orders_cummulative_b = orders_daily_b.apply(
    lambda x: orders_daily_b[orders_daily_b['date'] <= x['date']].agg(
        {'date': 'max', 'orders_daily_b': 'sum', 'revenue_per_date_b': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])

orders_cummulative_b.columns = ['date','orders_cummulative_b','revenue_cummulative_b']


data = (
    orders_daily_a.merge(
        orders_daily_b, left_on='date', right_on='date', how='left'
    )
    .merge(orders_cummulative_a, left_on='date', right_on='date', how='left')
    .merge(orders_cummulative_b, left_on='date', right_on='date', how='left')
    .merge(visitors_daily_a, left_on='date', right_on='date', how='left')
    .merge(visitors_daily_b, left_on='date', right_on='date', how='left')
    .merge(visitors_cummulative_a, left_on='date', right_on='date', how='left')
    .merge(visitors_cummulative_b, left_on='date', right_on='date', how='left')
)

data.head(6)

Создадим переменные orders_by_users_a и orders_by_users_a; в них для пользователей, которые заказывали хотя бы 1 раз, укажем число совершённых заказов.

In [ ]:
orders_by_users_a = (
    orders[orders['group'] == 'A']
    .groupby('visitor_id', as_index=False)
    .agg({'transaction_id': pd.Series.nunique})
)
orders_by_users_a.columns = ['visitor_id', 'orders']

orders_by_users_b = (
    orders[orders['group'] == 'B']
    .groupby('visitor_id', as_index=False)
    .agg({'transaction_id': pd.Series.nunique})
)
orders_by_users_b.columns = ['visitor_id', 'orders']

Объявим переменные sample_a и sample_b, в которых пользователям из разных групп будет соответствовать количество заказов. Тем, кто ничего не заказал, будут соответствовать нули. Это нужно, чтобы подготовить выборки к проверке критерием Манна-Уитни.

In [ ]:
sample_a = pd.concat([orders_by_users_a['orders'],
                      pd.Series(
                          0, 
                          index=np.arange(data['visitors_per_date_a'].sum() - 
                                          len(orders_by_users_a['orders'])), 
                          name='orders')],axis=0
                    )

sample_b = pd.concat([orders_by_users_b['orders'],
                      pd.Series(
                          0, 
                          index=np.arange(data['visitors_per_date_b'].sum() - 
                                          len(orders_by_users_b['orders'])), 
                          name='orders')],axis=0
                    )


Задаем функцию, в которой:
- задаем уровень значимости alpha=0.05,   
- применим критерий Манна-Уитни,   
- отформатируем p-value, округлив его до трёх знаков после запятой,    
- выведем относительный прирост конверсии группы B: конверсия группы B / конверсия группы A - 1, округлив до трёх знаков после запятой.

In [ ]:
# Функция для проверки гипотезы о равенстве групп data A и data B
def stat_significance(data_a, data_b):
    alpha = 0.05
    p_value = stats.mannwhitneyu(data_a, data_b)[1]
    print("P-value: {0:.3f}".format(p_value))

    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу: между группами есть разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать группы разными")
    
    print("Относительный прирост В к А: {0:.3%}".format(data_b.mean() / data_a.mean()-1))

In [ ]:
stat_significance(sample_a, sample_b)

По неочищенным данным различия в конверсии между группами есть.   
P-value = 0.011 меньше 0.05. Значит, нулевую гипотезу о том, что статистически значимых различий в конверсии между группами нет, отвергаем. Относительный выигрыш группы B равен 15.980%.

#### Cтатистическая значимость различий в среднем чеке заказа между группами по «сырым» данным

Посчитаем статистическую значимость различий в среднем чеке между группами по «сырым» данным — без удаления аномальных пользователей.

<p>Введем основную и альтернативные гипотезы:</p>

$\begin{equation*}
 \begin{cases}
   H_0 :\text{различий в среднем чеке между группами нет}\\
   H_1 :\text{различия в среднем чеке между группами есть}
 \end{cases}
\end{equation*}$

Уровень значимости: $\alpha = 0.05$ 


Так как в наших данных по средним чекам есть большие выбросы, воспользуемся критерием Манна - Уитни

In [ ]:
stat_significance(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'])  

#### Cтатистическая значимость различий в конверсии между группами по «очищенным» данным

Введем наш отфильтрованную таблицу orders. Мы принял, то что считаем аномальными 3 и более заказов, и заказы на сумму выше 53904. Узнаем, сколько всего аномальных пользователей методом shape().

In [ ]:
orders_by_users = (
    orders.groupby('visitor_id', as_index=False)
    .agg({'transaction_id': 'nunique'})
)

orders_by_users.columns =  ['visitor_id', 'orders']
many_orders = np.percentile(orders_by_users['orders'], 99)
expensive_orders = np.percentile(orders['revenue'], 99)


users_with_many_orders = pd.concat(
    [
        orders_by_users_a[orders_by_users_a['orders'] > many_orders]['visitor_id'],
        orders_by_users_b[orders_by_users_b['orders'] > many_orders]['visitor_id'],
    ],
    axis=0,
)


users_with_expensive_orders = orders[orders['revenue'] > expensive_orders]['visitor_id']


abnormal_users = (
    pd.concat([users_with_many_orders, users_with_expensive_orders], axis=0)
    .drop_duplicates()
    .sort_values()
)
display(abnormal_users.head(5))
abnormal_users.shape[0]

Всего 16 аномальных пользователей.    
Узнаем, как их действия повлияли на результаты теста. Посчитаем статистическую значимость различий в конверсии между группами теста по очищенным данным. Сначала подготовим выборки количества заказов по пользователям по группам теста.

In [ ]:
sample_a_filtered = pd.concat(
    [
        orders_by_users_a[
            np.logical_not(orders_by_users_a['visitor_id'].isin(abnormal_users))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitors_per_date_a'].sum() - len(orders_by_users_a['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

sample_b_filtered = pd.concat(
    [
        orders_by_users_b[
            np.logical_not(orders_by_users_b['visitor_id'].isin(abnormal_users))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitors_per_date_b'].sum() - len(orders_by_users_b['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

In [ ]:
stat_significance(sample_a_filtered, sample_b_filtered)

На очищенных данных разница в конверсии между группами есть,  а относительный прирост конверсии группы В отнистельно группы А увеличился на 18.921%. 

#### Cтатистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным

Сформулируем 2 стат.гипотезы: 
- H0: Средний чек  в группе A статистически НЕ ОТЛИЧАЕТСЯ от  среднего чека  в группе B
- Н1: Средний чек  в группе A статистически ОТЛИЧАЕТСЯ от среднего чека в группе B

In [ ]:
stat_significance(
    orders[(orders['group']=='A') & np.logical_not(orders['visitor_id'].isin(abnormal_users))]['revenue'], 
    orders[(orders['group']=='B') & np.logical_not(orders['visitor_id'].isin(abnormal_users))]['revenue']
                  )

   
 Значение P-значения (P-value) равное 0.788 говорит о том, что мы не можем отвергнуть нулевую гипотезу о том, что нет статистически значимых различий между группами A и B. Это означает, что на текущем уровне значимости (обычно 0.05) нет оснований полагать, что средние значения выборок A и B различаются.

Относительный прирост B к А, равный -3.234%, указывает на то, что значение в группе B меньше на 3.234% по сравнению с группой A относительно базового значения (группы A).

Таким образом, на основе данного теста мы не имеем статистически значимых оснований считать две группы разными.


#### Графики кумулятивной конверсии и среднего чека по очищенным данным

In [ ]:
stat_significance(
    orders[(orders['group']=='A') & np.logical_not(orders['visitor_id'].isin(abnormal_users))]['revenue'], 
    orders[(orders['group']=='B') & np.logical_not(orders['visitor_id'].isin(abnormal_users))]['revenue']
                  )

P-value значительно больше 0.05. Значит, причин отвергать нулевую гипотезу и считать, что в среднем чеке есть различия, нет. По разнице средних чеков групп различий практически нет.

На основании входных данных, предоставленных интернет-магазином был проведено исследование и вынесены рекомендации, изложенные ниже.
    
**1. В части приоритизации гипотез из списка, предоставленных отделом Маркетинга следует в первую очередь обратить внимание на гипотезы:**   
- "Запустить акцию, дающую скидку на товар в день рождения",   
- "Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей",   
- "Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок";      

Если ранжирование гипотез должно включать в себя и охват пользователей интернет-магазина, то места необходимо распределить таким образом:   
- "Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок",    
- "Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа",   
- "Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей". 	


**2. В части анализа А/В теста:**   

* График кумулятивной конверсии стабилизировался и при проверке гипотез ( и по сырым, и по очищенным данным) мы можем говорить, что конверсия в группе B лучше, чем в группе A
* График среднего чек не стабилизировался. График кумулятивного среднего чека по очищенным данным показывает, что график скачет и каждый день группа A и группа B меняются местами
* Есть статистически значимое различие по конверсии между группами как по «сырым», так и по данным после фильтрации аномалий. Конверсия группы В выше, чем в А.  
* Нет статистически значимого различия по среднему чеку между группами ни по «сырым», ни по данным после фильтрации аномалий. При этом средний чек группы В выше;   
* График относительного изменения кумулятивной конверсии группы B к группе A показывает, что результаты группы В стабильно лучше группы А;


Примем решение завершить тест и зафиксировать ,что "группа В показала себя лучше" (ее конверсия значительно  выше конверсии группы А). 



* Есть статистически значимое различие по конверсии между группами как по «сырым», так и по данным после фильтрации аномалий. Конверсия группы В выше, чем в А.  
* Нет статистически значимого различия по среднему чеку между группами ни по «сырым», ни по данным после фильтрации аномалий. При этом средний чек группы В выше;   
* График относительного изменения кумулятивной конверсии группы B к группе A показывает, что результаты группы В стабильно лучше группы А;


Примем решение завершить тест и зафиксировать ,что "группа В показала себя лучше" (ее конверсия значительно  выше конверсии группы А). 
 